<a href="https://colab.research.google.com/github/Fuenfgeld/2022TeamADataEngineeringBC/blob/main/Pr%C3%A4sentationen/02-Einfache%20Transformationen/%C3%9Cbungsaufgaben_Vortrag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Einfache Transformationen

In [61]:
# Bibliotheken einmalig installieren

%pip install pyspark pandas

In [ ]:
!wget https://raw.githubusercontent.com/Fuenfgeld/2022TeamADataEngineeringBC/main/Pr%C3%A4sentationen/02-Einfache%20Transformationen/create_data.py

--2022-05-12 12:35:07--  https://raw.githubusercontent.com/Fuenfgeld/2022TeamADataEngineeringBC/main/Pr%C3%A4sentationen/02-Einfache%20Transformationen/create_data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10138 (9.9K) [text/plain]
Saving to: ‘create_data.py’

create_data.py      100%[===================>]   9.90K  --.-KB/s    in 0s      

2022-05-12 12:35:08 (76.6 MB/s) - ‘create_data.py’ saved [10138/10138]



In [ ]:
import sqlite3
import pandas as pd
import pyspark.sql.functions as pysparkFunctions 
import os

os.system("python3 create_data.py")

0

In [ ]:
## (Py)Spark starten

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
## JSON-/CSV-Dateien in Dataframes schreiben
df_fru = spark.read.options(header='True', multiline='True', inferSchema='True').csv('Fruits.csv')
df_veg = spark.read.option("multiline",True).json('Vegetables.json')

# erstellte Dataframes ausgeben (10 Einträge ausgeben)
df_fru.show(10)
df_veg.show(10)


+------------+-----+----+-----------------+-------+-------------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|
+------------+-----+----+-----------------+-------+-------------+
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|   4|               14|      0|            0|
|strawberries|    7|   5|               14|      0|            5|
|strawberries|    7|   6|               18|     30|           10|
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|   7|               20|     60|           25|
+------------+-----+----+-----------------+-------+-------------+
only showing top 10 rows

+--------+-----+-------+-----------------+----+---

In [ ]:
## Spalten abfragen:	Jeremy will sich Übersicht über generellen Wasserverbrauch der Gemüsesorten verschaffen -> dazu begutachtet er die Spalte 'Wasserverbrauch'  
df_veg.select("crop", "water_consumption").show()

+--------+-----------------+
|    crop|water_consumption|
+--------+-----------------+
|zucchini|                6|
|  radish|                4|
|zucchini|                5|
|  radish|                4|
|zucchini|                6|
|  radish|                5|
|zucchini|                7|
|  radish|                3|
|zucchini|                7|
|  radish|                6|
|zucchini|                9|
|  radish|                7|
|zucchini|               10|
|  radish|                8|
|zucchini|               13|
|  radish|                8|
|zucchini|               12|
|  radish|             null|
|zucchini|                5|
|zucchini|             null|
+--------+-----------------+



In [ ]:
## Zeilen abfragen: Jeremy überlegt welche Pflanze er anbaut und schaut sich dafür die Zucchini genauer an

df_veg_lokal = df_veg.collect() # notwendig, falls man in einem verteilten Systen arbeitet
print(f"Informationen zu Zucchini in der 9. Zeile: {df_veg_lokal[8]}\n")

Informationen zu Zucchini in der 9. Zeile: Row(crop='zucchini', field=5, revenue=Row(net=0, tax=0), water_consumption=7, week=5, yield_per_sqm=5)



In [ ]:
## Spalten hinzufügen: Jeremy möchte die Einnahmen der Pflanzen pro Ertrag/m² ermitteln

df_fru_plus = df_fru.withColumn("lucrativeness", df_fru.revenue * df_fru.yield_per_sqm)
df_fru_plus.show()

+------------+-----+----+-----------------+-------+-------------+-------------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|lucrativeness|
+------------+-----+----+-----------------+-------+-------------+-------------+
|strawberries|    7|   1|               12|      0|            0|            0|
|strawberries|    7|   2|               10|      0|            0|            0|
|strawberries|    7|   3|               12|      0|            0|            0|
|strawberries|    7|   4|               14|      0|            0|            0|
|strawberries|    7|   5|               14|      0|            5|            0|
|strawberries|    7|   6|               18|     30|           10|          300|
|strawberries|    7|   1|               12|      0|            0|            0|
|strawberries|    7|   2|               10|      0|            0|            0|
|strawberries|    7|   3|               12|      0|            0|            0|
|strawberries|    7|   7|               

In [ ]:
## Spalten entfernen:   Um einen klareren Blick auf die Daten zu erhalten, entfernt Jeremy alle Spalten
## die nicht über den Wasserverbrauch aussagen

df_veg_drop = df_veg.drop(*('field', 'revenue', 'week'))
df_fru_drop = df_fru.drop(*('field', 'revenue', 'week'))

df_fru_drop.show()
df_veg_drop.show()

+------------+-----------------+-------------+
|        crop|water_consumption|yield_per_sqm|
+------------+-----------------+-------------+
|strawberries|               12|            0|
|strawberries|               10|            0|
|strawberries|               12|            0|
|strawberries|               14|            0|
|strawberries|               14|            5|
|strawberries|               18|           10|
|strawberries|               12|            0|
|strawberries|               10|            0|
|strawberries|               12|            0|
|strawberries|               20|           25|
|strawberries|               26|           25|
|strawberries|               24|           25|
|strawberries|               10|           25|
|strawberries|             null|            0|
+------------+-----------------+-------------+

+--------+-----------------+-------------+
|    crop|water_consumption|yield_per_sqm|
+--------+-----------------+-------------+
|zucchini|              

In [ ]:
#df_veg.show()

In [ ]:
## Um die beiden Dataframes kombinieren zu können müssen sie in das gleiche Format gebracht werden.
## Achtung hier verändert sich der semantische Sinn der Spalte. Dies ist hier egal, weil wir sie weiter unten löschen.
df_veg_correct_revenue = df_veg.withColumn("revenue", pysparkFunctions.column("revenue.net"))
#df_veg_correct_revenue.show()


## DataFrames Zusammenfügen: Jeremy überlegt ob er Gemüse oder Obst anbauen soll, 
## deshalb verschafft er sich eine Gesamtübersicht/Vergleich aller Obst- & Pflanzensorten: Obst- & Gemüsesorten konkatenieren

df_fru_veg = df_fru.union(df_veg_correct_revenue)
df_fru_veg.show()

+------------+-----+----+-----------------+-------+-------------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|
+------------+-----+----+-----------------+-------+-------------+
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|   4|               14|      0|            0|
|strawberries|    7|   5|               14|      0|            5|
|strawberries|    7|   6|               18|     30|           10|
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|   7|               20|     60|           25|
|strawberries|    7|   8|               26|    150|           25|
|strawberries|    7|   9|               24|    150|           25|
|strawberr

In [ ]:
## Dupliakte entfernen: Wie Jeremy gerade aufgefallen ist, enthällt die Tabelle noch Duplikate.
## (Siehe z.B. Woche 2 & 8 der Erdbeeren) Diese sollen nun entfernt werden.

df_fru_veg.dropDuplicates().show()

+------------+-----+----+-----------------+-------+-------------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|
+------------+-----+----+-----------------+-------+-------------+
|strawberries|    7|   5|               14|      0|            5|
|strawberries|    7|   7|               20|     60|           25|
|strawberries|    7|   9|               24|    150|           25|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   6|               18|     30|           10|
|strawberries|    7|   8|               26|    150|           25|
|strawberries|    7|  11|             null|    150|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|  10|               10|    100|           25|
|strawberries|    7|   4|               14|      0|            0|
|strawberries|    7|   1|               12|      0|            0|
|    zucchini|    5|   0|                7|      5|            5|
|      rad

In [ ]:
## Aufräumen:	Für Jeremy sind jetzt natürlich nur Einträge relevant, die einen Wasserverbrauch beinhalten.

df_fru_veg = df_fru_veg.dropna()
df_fru_veg.show()

+------------+-----+----+-----------------+-------+-------------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|
+------------+-----+----+-----------------+-------+-------------+
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|   4|               14|      0|            0|
|strawberries|    7|   5|               14|      0|            5|
|strawberries|    7|   6|               18|     30|           10|
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|   7|               20|     60|           25|
|strawberries|    7|   8|               26|    150|           25|
|strawberries|    7|   9|               24|    150|           25|
|strawberr

In [ ]:
## Wasserverbrauch anzeigen:    Jeremy möchte wissen, wie der gesamte Wasserverbrauch im letzten Jahr war. 
## Und wie der Wasserverbrauch bezogen auf die jeweilige Nutzpflanze ausfällt.

df_fru_veg.agg(pysparkFunctions.sum("water_consumption")).show()

df_fru_veg.groupby('crop').sum("water_consumption").show()

df_fru_veg.filter(df_fru_veg.crop == "strawberries").agg(pysparkFunctions.sum("water_consumption")).show()

+----------------------+
|sum(water_consumption)|
+----------------------+
|                   319|
+----------------------+

+------------+----------------------+
|        crop|sum(water_consumption)|
+------------+----------------------+
|strawberries|                   194|
|    zucchini|                    80|
|      radish|                    45|
+------------+----------------------+

+----------------------+
|sum(water_consumption)|
+----------------------+
|                   194|
+----------------------+



**Übungen**

1. Dieses Notebook enthält noch eine weitere JSON Datei namens `Corn.json` mit Daten zu Gedreide. Werte den Wasserverbrauch nur für das Gedreide aus.
2. Ermittle wie viel Wasser pro Feld eingesetzt wurde.
3. Ermittle den gesamten Wasserverbrauch für alle Pfanzen.